In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

from numpy import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from pymystem3 import Mystem
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

D:\OPP\compVision\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
testSaved = pd.read_csv('testSaved.csv')
testSaved = testSaved.sample(frac=1).reset_index(drop=True)
testSaved

,Review,Status
0,"\n В Шоколаднице Боровское, 36А, бы...",0
1,"\n Люблю ""Шоколадницу"" за:\n- кожан...",1
2,\n И почему во время беременности т...,1
3,\n 6 марта 2014 года заказали доста...,0
4,\n Меню ресторана на любителя. В ме...,1
...,...,...
1995,\n Кофе to go приходится ждать по 1...,0
1996,\n На фоне Якитории и Планеты-Суши ...,1
1997,"\n Вкусно вкусно, все очень вкусно....",1
1998,\n Еда в заведении далека от того ч...,0


In [4]:
corp = []
for i in range(len(testSaved.index)):
    report = re.sub("[^А-я]", " ", str(testSaved["Review"][i]))
#     report = Mystem.lemmatize(str(dataset["Report"][i]).lower())
    report = report.lower()
    report = report.split()
    ps = PorterStemmer()
    report = [ps.stem(word) for word in report if not word in set(stopwords.words("russian"))]
    report = " ".join(report)
    corp.append(report)

In [8]:
import pickle
savedVocab = open("savedVocab.pkl", "rb")
vocab = pickle.load(savedVocab)
vocab

{'знаете': 4956,
 'насколько': 8043,
 'дорогие': 3218,
 'капсулы': 5760,
 'кофемашины': 6734,
 'шоколаднице': 19576,
 'купить': 6971,
 'капсул': 5759,
 'разными': 12864,
 'вкусами': 1518,
 'реально': 13258,
 'классные': 6125,
 'вкусы': 1600,
 'попробуйте': 10546,
 'обязательно': 8833,
 'попробовали': 10543,
 'новую': 8532,
 'систему': 14468,
 'заказа': 4234,
 'очередной': 9520,
 'макдональдс': 7340,
 'приятно': 11658,
 'удивил': 17561,
 'сбербанку': 13953,
 'россии': 13641,
 'десять': 2635,
 'дней': 2856,
 'веселья': 1284,
 'подарил': 9998,
 'бар': 301,
 'явно': 19913,
 'вспомнить': 1859,
 'ближайшие': 481,
 'дни': 2859,
 'обожаю': 8702,
 'шишас': 19519,
 'последней': 10663,
 'капли': 5749,
 'последнего': 10661,
 'коктейля': 6271,
 'говорил': 2186,
 'вреде': 1817,
 'пищи': 9856,
 'мака': 7316,
 'читайте': 19306,
 'таблицы': 16444,
 'соблюдать': 15031,
 'суточные': 16195,
 'нормы': 8569,
 'нормально': 8559,
 'всем': 1849,
 'привет': 11126,
 'давно': 2432,
 'пользуюсь': 10421,
 'доставко

In [9]:
w1 = len(vocab)
s1 = len(corp)
testMatrix = np.zeros((s1, w1))

for i in range(s1):
    senLine = corp[i].split(" ")
    for word in senLine:
        if word!='' and word in vocab.keys():
            testMatrix[i][vocab[word]] += 1

In [10]:
test = testSaved
test['Review'] = testMatrix.tolist()
listOfLists = test['Review'].values.tolist()
testX = tf.constant(listOfLists)
testY = test.iloc[:, 1].values
testX

<tf.Tensor: shape=(2000, 20000), dtype=float32, numpy=
array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [12]:
reconstructed_model = keras.models.load_model("savedModel")

In [13]:
score = reconstructed_model.evaluate(testX, testY)

63/63 [==============================] - 1s 6ms/step - loss: 0.7386 - accuracy: 0.8500


In [34]:
preds = reconstructed_model.predict(testX, batch_size=None, verbose=0, steps = None)
test_prediction_label = np.argmax(preds.T, axis = 0)
pred_test_labels = []
for row in test_prediction_label:
        pred_test_labels.append(row)

data = {'Predicted':pred_test_labels,
        'Actual':testY}
compare = pd.DataFrame(data)
test_prediction_label

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# #     print(compare)

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [38]:
confusion_matrix(testY, test_prediction_label) 

array([[ 680,  153],
       [ 147, 1020]], dtype=int64)

In [17]:
print(classification_report(test_prediction_label, testY))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       827
           1       0.87      0.87      0.87      1173

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



In [28]:
test = reconstructed_model.predict(testX);
test_prediction_label = np.argmax(test.T, axis = 0)
test_prediction_label

array([1, 1, 0, ..., 1, 0, 0], dtype=int64)

In [27]:
preds = reconstructed_model.predict(testX, batch_size=None, verbose=0, steps = None)
test_prediction_label = np.argmax(preds.T, axis = 0)
pred_test_labels = []
for row in test_prediction_label:
        pred_test_labels.append(row)

data = {'Predicted':pred_test_labels,
        'Actual':testY}
compare = pd.DataFrame(data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(preds)

[[4.2184851e-01 6.3476038e-01]
 [1.1005104e-03 9.9879193e-01]
 [9.6034193e-01 5.2246541e-02]
 ...
 [4.0340446e-08 1.0000000e+00]
 [9.7808313e-01 3.2241404e-02]
 [9.9830997e-01 3.3062994e-03]]
